In [1]:
# 📦 Imports
import os
import json
import torch
import wandb
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

In [2]:
# 🧪 Init W&B
wandb.init(
    project="cardiovascular-expert-sft",
    name="tinyllama-cardio-expert-v1",
    tags=["tinyllama", "sft", "cardiovascular", "medical"],
    notes="SFT of TinyLlama for cardiovascular expertise"
)
#key 2184af33313777a95ce10dc38c2e90fd3a202bdd

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dyh2111 (med-moe) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
# 💾 Save path
model_path = 'moeme/model/cardiovascular_expert_model'

# 🔧 Hyperparameters
wandb_config = {
    "model_name": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "learning_rate": 2e-4,
    "epochs": 20,
    "batch_size": 4,
    "gradient_accumulation_steps": 8,
    "lora_r": 16,
    "lora_alpha": 32,
    "medical_domain": "cardiovascular",
    "load_pretrained": True  # Set to False to load model from scratch
}
wandb.config.update(wandb_config)

In [4]:
# 🧠 Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [5]:
# 🧾 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(wandb_config["model_name"])
tokenizer.pad_token = tokenizer.eos_token


In [6]:
# 🧠 Model
if wandb_config["load_pretrained"] and os.path.exists(model_path):
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map="auto",
        quantization_config=bnb_config,
        torch_dtype=torch.float16,
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        wandb_config["model_name"],
        device_map="auto",
        quantization_config=bnb_config,
        torch_dtype=torch.float16,
    )

model = prepare_model_for_kbit_training(model)

In [7]:
# 🧪 LoRA Config
lora_config = LoraConfig(
    r=wandb_config["lora_r"],
    lora_alpha=wandb_config["lora_alpha"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)
model = get_peft_model(model, lora_config)

In [8]:
# 📂 Load Dataset
from prompt_template import prompt_template

with open("blood_heart_circulation_qa.json", "r") as f:
    qa_data = json.load(f)

train_data = []
for topic in qa_data:
    for question, answer in topic['question_answer_pair']:
        prompt = prompt_template(question, answer, benchmark='pubmedqa')
        train_data.append({"text": prompt})

dataset = Dataset.from_list(train_data).train_test_split(test_size=0.1)

In [15]:
# 🔁 Tokenize
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"][:]
    return tokens

tokenized = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/278 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [16]:
# ⚙️ Training Args
training_args = TrainingArguments(
    output_dir=model_path,
    per_device_train_batch_size=wandb_config["batch_size"],
    per_device_eval_batch_size=wandb_config["batch_size"],
    gradient_accumulation_steps=wandb_config["gradient_accumulation_steps"],
    num_train_epochs=wandb_config["epochs"],
    learning_rate=wandb_config["learning_rate"],
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    report_to="wandb"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:
# 🚀 Train
trainer.train()
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

/home/dyh2111/moeme/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,5.203900
20,0.596600
30,0.462100
40,0.445100
50,0.399400
60,0.357100
70,0.324200
80,0.278900
90,0.233700
100,0.201800


('moeme/model/cardiovascular_expert_model/tokenizer_config.json',
 'moeme/model/cardiovascular_expert_model/special_tokens_map.json',
 'moeme/model/cardiovascular_expert_model/tokenizer.model',
 'moeme/model/cardiovascular_expert_model/added_tokens.json',
 'moeme/model/cardiovascular_expert_model/tokenizer.json')

# For continued training

In [ ]:
# 🧠 Reload model from saved path for continued training
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# 🧪 Restart W&B run
wandb.init(
    project="cardiovascular-expert-sft",
    name="tinyllama-cardio-expert-v2-continued",
    tags=["tinyllama", "sft", "continued-training"],
    notes="Continued SFT with same dataset but new W&B run"
)

# 🛠️ You can modify learning rate, batch size, or epochs here
training_args = TrainingArguments(
    output_dir=model_path,
    per_device_train_batch_size=wandb_config["batch_size"],
    per_device_eval_batch_size=wandb_config["batch_size"],
    gradient_accumulation_steps=wandb_config["gradient_accumulation_steps"],
    num_train_epochs=10,  # Continued for 10 more epochs
    learning_rate=1e-4,   # Optional new LR
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    save_total_limit=2,
    bf16=True,
    report_to="wandb",
    load_best_model_at_end=True,
)

# 🔁 Continue Training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
)

trainer.train()
trainer.save_model(model_path)
